In [23]:
'''用策略迭代解决冰湖问题'''
'''问题描述见readme.md
  求解目标：：智能体学习从起始块如何行动到目标块，并避开危险块。即确
定智能体的最优行动路线。'''

#导入解决该问题需要的库(基于Pytorch环境)
import numpy as np
import gym

#利用gym，创建冰湖环境
env=gym.make('FrozenLake-v0')

#设置最大迭代次数
Iterations=1000000
# set the threshold
#设置误差，以判断策略迭代是否已经收敛
Epsilon=1e-8
#设置折扣因子
Gama=1.0

#通过贝尔曼方程计算价值函数
def Value_Function(Policy,Gama):
    #初始化价值表
    Value_Table=np.zeros(env.nS)

    while True:
        #更新价值表
        New_Value_Table=np.copy(Value_Table)
        #在当前状态下，根据策略选择动作并计算价值表
        for i in range(env.nS):
            a=Policy[i]
            #计算价值表
            Value_Table[i] = sum([trans_prob * (reward_prob + Gama * New_Value_Table[next_state])
                        for trans_prob, next_state, reward_prob, _ in env.P[i][a]])
            #Value_Table[i]=sum([(reward_prob+Gama*New_Value_Table[next_state])*trans_prob]) for trans_prob,next_state,reward_prob,_ in env.P[i][a]])

                

        if(np.sum((np.fabs(New_Value_Table - Value_Table))) <= Epsilon):
            break
    return Value_Table

#制定策略
def Get_Policy(Value_Table,Gama):
    Policy=np.zeros(env.observation_space.n)
    for i in range(env.observation_space.n):
        #构建并初始化Q表
        Q=np.zeros(env.action_space.n)
        #计算所有动作的Q值
        for a in range(env.action_space.n):
            for n in env.P[i][a]:
                trans_prob, next_state, reward_prob, _ = n
                Q[a]=Q[a]+(trans_prob * (reward_prob + Gama * Value_Table[next_state]))
        #通过贝尔曼最优方程得到最佳动作
        Policy[i]=np.argmax(Q)
    return Policy

def Policy_Itreration(env,Gama):
    Gama=1.0
    #初始化
    O_Policy=np.zeros(env.observation_space.n)
    #开始迭代并检查是否已收敛
    for i in range(Iterations):
        #价值函数计算
        Value_Fun=Value_Function(O_Policy,Gama)
        #新策略
        N_Policy=Get_Policy(Value_Fun,Gama)
        #检查是否已收敛
        if(np.all(O_Policy==N_Policy)):
            print('迭代次数为%d'%(i+1))
            break
        O_Policy=N_Policy
    return N_Policy
    
#输出策略
print('得到的策略如下')
print(Policy_Itreration(env,Gama))

迭代次数为7
[0. 3. 3. 3. 0. 0. 0. 0. 3. 1. 0. 0. 0. 2. 1. 0.]
